# Bader Charge

For the source code, see [bader](https://workgraph-collections.readthedocs.io/en/latest/qe/module.html#workgraph_collections.ase.espresso.bader.bader_workgraph).

## Example: H2O molecule

### Prepare the inputs and submit the workflow


In [1]:
from aiida import load_profile
from aiida.orm import Dict, KpointsData, StructureData, load_code
from aiida_pseudo.data.pseudo import UpfData
from ase.build import molecule
from workgraph_collections.qe.bader import BaderWorkgraph


load_profile()
#===============================================================================
# load the codes
pw_code = load_code("pw-7.4@localhost")
pp_code = load_code("pp-7.4@localhost")
bader_code = load_code("bader@localhost")
# ===============================================================================
# create input structure
atoms = molecule("H2O")
atoms.center(vacuum=3.0)
atoms.pbc = True
structure = StructureData(ase=atoms)
# create input parameters node
scf_paras = Dict(
    {
        "CONTROL": {
            "calculation": "scf",
        },
        "SYSTEM": {
            "ecutwfc": 30,
            "ecutrho": 240,
            "occupations": "smearing",
            "smearing": "gaussian",
            "degauss": 0.1,
        },
    }
)
kpoints = KpointsData()
kpoints.set_kpoints_mesh([1, 1, 1])
# Load the pseudopotential kjpaw.
pseudos = {"H": UpfData("/home/xing/data/ase/espresso_pseudo/kjpaw/H.pbe-kjpaw_psl.1.0.0.UPF"),
           "O": UpfData("/home/xing/data/ase/espresso_pseudo/kjpaw/O.pbe-n-kjpaw_psl.1.0.0.UPF")}
#
#
metadata = {
    "options": {
        "resources": {
            "num_machines": 1,
            "num_mpiprocs_per_machine": 1,
        },
    }
}
# ===============================================================================
inputs = {"scf":{
                "pw": {
                    "parameters": scf_paras,
                    "pseudos": pseudos,
                    "metadata": metadata
                },
                "kpoints": kpoints,
            },

        "pp_valence": {
                "parameters": Dict({
                        "INPUTPP": {"plot_num": 0},
                        "PLOT": {"iflag": 3},
                    }),
                "metadata": metadata,
        },
        "pp_all": {
                "parameters": Dict({
                        "INPUTPP": {"plot_num": 21},
                        "PLOT": {"iflag": 3},
                    }),
                "metadata": metadata,
        },
        "bader": {
            "metadata": metadata,
        }
}
# prepare inputs and submit
wg = BaderWorkgraph.build(structure=structure, pw_code=pw_code,
                     pp_code=pp_code, bader_code=bader_code,
                     inputs=inputs)
wg.to_html()

Run the WorkGraph:


In [2]:
wg.run()

#------------------------- Print the output -------------------------
charges = wg.outputs.bader_charge.value.get_array("charge")
print("Bader charges:")
print("Index    Symbol    Charge")
for i, charge in enumerate(charges):
    print(f"{i:5d}    {atoms.get_chemical_symbols()[i]:5s}    {charge:5.3f}")


10/08/2025 11:12:15 AM <2502881> aiida.orm.nodes.process.workflow.workchain.WorkChainNode: [REPORT] [201776|WorkGraphEngine|continue_workgraph]: tasks ready to run: PwBaseWorkChain
10/08/2025 11:12:15 AM <2502881> aiida.orm.nodes.process.workflow.workchain.WorkChainNode: [REPORT] [201776|WorkGraphEngine|on_wait]: Process status: Waiting for child processes: 201779
10/08/2025 11:12:17 AM <2502881> aiida.orm.nodes.process.workflow.workchain.WorkChainNode: [REPORT] [201779|PwBaseWorkChain|run_process]: launching PwCalculation<201782> iteration #1
10/08/2025 11:12:28 AM <2502881> aiida.orm.nodes.process.workflow.workchain.WorkChainNode: [REPORT] [201779|PwBaseWorkChain|results]: work chain completed after 1 iterations
10/08/2025 11:12:29 AM <2502881> aiida.orm.nodes.process.workflow.workchain.WorkChainNode: [REPORT] [201779|PwBaseWorkChain|on_terminated]: remote folders will not be cleaned
10/08/2025 11:12:29 AM <2502881> aiida.orm.nodes.process.workflow.workchain.WorkChainNode: [REPORT] [

Bader charges:
Index    Symbol    Charge
    0    O        7.147
    1    H        0.427
    2    H        0.427
